In [81]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report
from pandas import DataFrame
from pandas import Series
from scipy.stats import linregress
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import warnings
import csv
import os 

In [82]:
test_df = pd.read_csv('test.csv')
credit_df = pd.read_csv('credit_risk_dataset.csv')
credit_scoring_df = pd.read_csv('credit_scoring.csv')

In [83]:
#Rename the test dateFrame columns: Credit_Utilization_Ratio, Interest_Rate, Num_of_Loan, Annual_Income
test_df.rename(columns = {'Credit_Utilization_Ratio':'Credit Utilization Ratio',
                         'Interest_Rate':'Interest Rate',
                         'Type_of_Loan':'Loan Intent',
                         'Num_of_Loan':'Loan Accounts',
                         'Annual_Income':'Annual Income',                          
                         }, inplace=True)

#Rename credit risk columns: person_income, loan_int_rate, loan_intent
credit_df.rename(columns = {'person_income':'Annual Income',
                           'person_age':'Age',
                            'loan_int_rate':'Interest Rate',
                           'loan_intent':'Loan Intent',                 
                           }, inplace=True)

#Rename credit risk columns: person_income, Education Level, Type of Loan
credit_scoring_df.rename(columns = {'Education Level':'Occupation',
                                    'Type of Loan':'Loan Intent',
                                   }, inplace=True)

In [84]:
test_dfc = test_df
credit_dfc = credit_df
credit_scoring_dfc = credit_scoring_df

In [85]:
test_dfc = test_df.drop(columns=['ID','Customer_ID', 'Month','Loan Accounts','Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Delay_from_due_date','Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt', 'Credit_History_Age', 'Payment_of_Min_Amount', 'Total_EMI_per_month', 'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance'])
credit_dfc = credit_df.drop(columns=['person_home_ownership','person_emp_length','loan_grade','loan_amnt','loan_status','cb_person_default_on_file','cb_person_cred_hist_length'])
credit_scoring_dfc = credit_scoring_df.drop(columns=['Gender','Marital Status','Payment History','Number of Credit Accounts','Loan Amount','Loan Term'])

In [86]:
#merge the DataFrames
half_concat = pd.concat([credit_dfc,test_dfc], axis=0)
full_concat = pd.concat([half_concat,credit_scoring_dfc], axis=0)

In [87]:
full_concat = full_concat.dropna(subset=['Credit Utilization Ratio'])
full_concat

,Age,Annual Income,Loan Intent,Interest Rate,loan_percent_income,Name,SSN,Occupation,Credit Utilization Ratio,Employment Status
0,23,19114.12,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3.00,NaN,Aaron Maashoh,821-00-0265,Scientist,35.030402,NaN
1,24,19114.12,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3.00,NaN,Aaron Maashoh,821-00-0265,Scientist,33.053114,NaN
2,24,19114.12,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3.00,NaN,Aaron Maashoh,821-00-0265,Scientist,33.811894,NaN
3,24,19114.12,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3.00,NaN,Aaron Maashoh,821-00-0265,Scientist,32.430559,NaN
4,28,34847.84,Credit-Builder Loan,6.00,NaN,Rick Rothackerj,004-07-5839,_______,25.926822,NaN
...,...,...,...,...,...,...,...,...,...,...
995,59,NaN,Auto Loan,12.99,NaN,NaN,NaN,High School,0.740000,Employed
996,64,NaN,Home Loan,18.02,NaN,NaN,NaN,Bachelor,0.770000,Unemployed
997,63,NaN,Personal Loan,18.95,NaN,NaN,NaN,Master,0.180000,Self-Employed
998,51,NaN,Auto Loan,1.80,NaN,NaN,NaN,PhD,0.320000,Self-Employed


In [89]:
full_concat.columns

Index(['Age', 'Annual Income', 'Loan Intent', 'Interest Rate',
       'loan_percent_income', 'Name', 'SSN', 'Occupation',
       'Credit Utilization Ratio', 'Employment Status'],
      dtype='object')

In [90]:
full_group = full_concat.groupby(['Age','Annual Income','Loan Intent','Interest Rate','Name','Occupation' ])['Credit Utilization Ratio'].mean().reset_index()

In [91]:
full_group

,Age,Annual Income,Loan Intent,Interest Rate,Name,Occupation,Credit Utilization Ratio
0,-500,10077.135,"Personal Loan, Mortgage Loan, and Payday Loan",27.0,Cheet,_______,21.906967
1,-500,100914.39,"Home Equity Loan, Home Equity Loan, Debt Conso...",16.0,ernie Woodallm,Manager,32.059348
2,-500,10120.28,"Debt Consolidation Loan, Home Equity Loan, Aut...",33.0,Maloner,Doctor,40.543923
3,-500,10143.89,"Payday Loan, Debt Consolidation Loan, and Mort...",14.0,Kiharap,Developer,37.304868
4,-500,101607.27,"Student Loan, Personal Loan, Student Loan, and...",10.0,Laurenh,Doctor,26.434860
...,...,...,...,...,...,...,...
21563,985,33494.58,"Not Specified, Student Loan, and Payday Loan",12.0,Andyw,Entrepreneur,33.351790
21564,989,21733.845,"Payday Loan, Debt Consolidation Loan, and Debt...",6.0,Garima Goelo,Architect,33.024684
21565,993,70732.4,"Personal Loan, Not Specified, Debt Consolidati...",26.0,Eveline Danubratae,Scientist,27.243809
21566,996,24482.38,Student Loan,9.0,Ernest Scheyderi,Scientist,32.599270


In [92]:
full_concat.dtypes

Age                          object
Annual Income                object
Loan Intent                  object
Interest Rate               float64
loan_percent_income         float64
Name                         object
SSN                          object
Occupation                   object
Credit Utilization Ratio    float64
Employment Status            object
dtype: object

In [101]:
#Credit utilization of lesser than .30 , Optimal credit utilization
filtered_credit_scoring_df = full_concat.loc[full_concat["Credit Utilization Ratio"]<=.99]
filtered_credit_scoring_df

,Age,Annual Income,Loan Intent,Interest Rate,loan_percent_income,Name,SSN,Occupation,Credit Utilization Ratio,Employment Status
0,60,NaN,Personal Loan,2.65,NaN,NaN,NaN,Master,0.22,Employed
1,25,NaN,Auto Loan,5.19,NaN,NaN,NaN,High School,0.20,Unemployed
2,30,NaN,Auto Loan,2.76,NaN,NaN,NaN,Master,0.22,Employed
3,58,NaN,Auto Loan,6.57,NaN,NaN,NaN,PhD,0.12,Unemployed
4,32,NaN,Personal Loan,6.28,NaN,NaN,NaN,Bachelor,0.99,Self-Employed
...,...,...,...,...,...,...,...,...,...,...
995,59,NaN,Auto Loan,12.99,NaN,NaN,NaN,High School,0.74,Employed
996,64,NaN,Home Loan,18.02,NaN,NaN,NaN,Bachelor,0.77,Unemployed
997,63,NaN,Personal Loan,18.95,NaN,NaN,NaN,Master,0.18,Self-Employed
998,51,NaN,Auto Loan,1.80,NaN,NaN,NaN,PhD,0.32,Self-Employed


In [102]:
# Initialize a list to store account records
account_records = []

# Iterate over each row in the DataFrame using a for loop
for index, row in full_concat.iterrows():
    account_record = {
        'Age': row['Age'],
        'Annual Income': row['Annual Income'],
        'Loan Intent': row['Loan Intent'],
        'Interest Rate': row['Interest Rate'],	
        'loan_percent_income': row['loan_percent_income'],	
        'Name': row['Name'],	
        'Occupation': row['Occupation'],	
        'Credit Utilization Ratio': row['Credit Utilization Ratio'],	
        'Employment Status': row['Employment Status'],
        # Add more fields as necessary
    }
    account_records.append(account_record)

In [103]:
account_records

[{'Age': '23',
  'Annual Income': '19114.12',
  'Loan Intent': 'Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan',
  'Interest Rate': 3.0,
  'loan_percent_income': nan,
  'Name': 'Aaron Maashoh',
  'Occupation': 'Scientist',
  'Credit Utilization Ratio': 35.03040186,
  'Employment Status': nan},
 {'Age': '24',
  'Annual Income': '19114.12',
  'Loan Intent': 'Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan',
  'Interest Rate': 3.0,
  'loan_percent_income': nan,
  'Name': 'Aaron Maashoh',
  'Occupation': 'Scientist',
  'Credit Utilization Ratio': 33.0531145,
  'Employment Status': nan},
 {'Age': '24',
  'Annual Income': '19114.12',
  'Loan Intent': 'Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan',
  'Interest Rate': 3.0,
  'loan_percent_income': nan,
  'Name': 'Aaron Maashoh',
  'Occupation': 'Scientist',
  'Credit Utilization Ratio': 33.81189412,
  'Employment Status': nan},
 {'Age': '24',
  'Annual Income': '19114.12',
  'Loan